In [1]:
# Now I need to recalculate lake metadata (weights ect) in the same manner as the catchments
import ECCO_functions_v2 as ECCO

import pandas as pd
import osgeo.ogr
import sys, time, os, json, glob
import numpy as np
import pyproj
import h5py
import csv  # <<-- dont forget to add this ti function file
import random

import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
def Lake_surfaceweights_meta(nc_path,sbar=False):
    '''
    Purpose:          
    This program Generates metadata files used to speed up the final runs.
    This was forthe lakes (not catchments), and created a metadata text
    file, to be used as a mini-database in pandas.
    This is the second version if of a program to generate lake surface 
    weights and metadata. This was made after the program to do the same
    for the catchments. This creates hdf5 weight file, and .csv metadata.
    '''
        
    clim_dat,rlat,rlon,time,metadata,txtfname = ECCO.Read_CORDEX_V2(nc_path)
    vname, m1, m2, dexp, m3, m4, m5, m6, drange_orignial = metadata 
 
    var_type = clim_dat.standard_name       # What kind of CORDEX data?
    dat_loaded = clim_dat[0,:,:]            # Load CORDEX data into RAM
    print np.shape(dat_loaded)
    rlat_loaded = rlat[:]
    rlon_loaded = rlon[:]

    lake_file = 'Lakes/ecco-biwa_lakes_v.0.2.shp'
    
    thefilename = 'lake_weights'
    FILE= 'Lakes/Weights/' + thefilename +'.h5'                # Set up HDF5 file output
    if os.path.isfile(FILE):
        #print 'HDF5 File already exists. Leaving loop so you dont clobber it by accident.'
        #print 'To run this function, decide manually if you want to remove it or not.'
        #return
        print 'hdf weights file exists, removing it...'
        os.remove(FILE)
    else:
        print 'Creating file: ',FILE
    fweights = h5py.File(FILE,'w')
    
    # set and write header info for the metadata file
    metacsv = 'Lakes/Metadata/Lake_meta.csv'
    if os.path.isfile(metacsv) == True:
        print 'Earlier metadata exists. Erasing it...'
        os.remove(metacsv)
    tmplist = ['num','EB_id','area','npix','ypix','xpix'] 
    ECCO.write_metadata_csv(mfname=metacsv,meta_list=tmplist)
    
    #orog = ECCO.Height_CORDEX()
    ShapeData = osgeo.ogr.Open(lake_file)
    TheLayer = ShapeData.GetLayer(iLayer=0)
    dolakes=range(TheLayer.GetFeatureCount())

    if sbar:
        icnt = 0
    
    for num in dolakes[0:1000]:
        feature1 = TheLayer.GetFeature(num) 
        lake_feature = feature1.ExportToJson(as_object=True)
        lake_cart = ECCO.Path_LkIsl_ShpFile(lake_feature['geometry']['coordinates']) 
        EB_id = lake_feature['properties']['EBhex'][2:]
        lake_altitude=lake_feature['properties']['vfp_mean']

        lake_rprj = ECCO.Path_Reproj(lake_cart,False)

        sub_clim,sub_rlat,sub_rlon = ECCO.TrimToLake(lake_rprj,dat_loaded,rlat_loaded,
                                                        rlon_loaded,off = 3, show = False) 
        weight_mask = ECCO.Pixel_Weights(lake_rprj,sub_clim,sub_rlat,sub_rlon)


        pix_truth = (weight_mask > 0.0)      # Count how many 
        pxnum = len(weight_mask[pix_truth])  #  pixels of data are needed.
    
        ypix = -99
        xpix = -99
        if pxnum == 1:
            xxx,yyy = ECCO.Get_LatLonLim(lake_rprj.vertices)  # Find upp./low.lake lims.
            ypix = (ECCO.Closest(rlat,yyy[0]))                # For lakes of one pixel  
            xpix = (ECCO.Closest(rlon,xxx[0]))
        if pxnum < 1:
            pxnum = 1  # Small bug where it thinks lakes dont exist, no biggy...
        if pxnum > 1:
            ECCO.Write_HDF_weights(fw=fweights,EB_id=EB_id,weights=weight_mask) 
        
        tmpmeta =[num,EB_id, ECCO.Area_Lake_and_Islands(lake_cart),pxnum,ypix,xpix]
        ECCO.write_metadata_csv(mfname=metacsv,meta_list=tmpmeta)
        
        #print num,EB_id, ECCO.Area_Lake_and_Islands(lake_cart),pxnum,ypix,xpix
        if sbar:
            icnt=icnt+1
            if (float(icnt) % 10.) == 0.0:
                ECCO.Update_Progress(float(icnt)/len(dolakes))
    fweights.close()
    return 

In [37]:
#ncfile= '/uio/kant/geo-metos-u1/blaken/datadisk/ECCO/CORDEX/Data_CORDEX/tas_EUR-11_ICHEC-EC-EARTH_historical_r1i1p1_KNMI-RACMO22E_v1_day_19660101-19701231.nc'
ncfile ='CORDEX/tas_EUR-11_ICHEC-EC-EARTH_rcp45_r1i1p1_KNMI-RACMO22E_v1_day_20960101-21001231.nc'

Lake_surfaceweights_meta(nc_path=ncfile,sbar=True)

(412, 424)
hdf weights file exists, removing it...
Earlier metadata exists. Erasing it...
Progress: [----------------------------------------] 0% 

In [21]:
lake_meta = pd.read_csv('Lakes/Metadata/Lake_meta.csv')
lake_meta = lake_meta.set_index('EB_id')
lake_meta.index[1]

'ccc59c'

In [24]:
#lake_meta

In [29]:
# Example of reading the data from a hdf5 file after creation using the metdatada as a lookup
#filein = 'Catchments/Weights/catchment_weights.h5'
#f = h5py.File(filein,"r")
# Going to test the weighting for the lakes (.npy) files, and also for the catchment hdf5 file.
# will read them in a loop, and sum the pixels. Will stop if the weight masks are not equal to
# 1 at any time.

acnt = 0
for n in lake_meta.index:
    if lake_meta.npix[n] > 1:
        with h5py.File('Lakes/Weights/lake_weights.h5','r') as fp:
            tmparray = fp[n]
            tmparray = tmparray[:,:]
        if tmparray.sum() > 1.01:
            acnt += 1.
            print n, tmparray.sum()
print 'Found problems in', acnt,' lakes'

# Found problems in 2026.0  lakes: too little weight
# Found problems in 2072.0  lakes: too much weight
# After change: Found problems in 0  lakes

599f21 1.01356
854aec 1.01046
1ccbad 1.02334
a96b25 1.01708
60bfab 1.01972
Found problems in 5.0  lakes
